In [127]:
#Контрольная работа "SparkSQL"
#выполнялась в jn на образе jupiter/pyspark-notebook 

from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import Window as w

spark = SparkSession.builder.master('local').getOrCreate()
sc = spark.sparkContext

df = (spark.read.option('header', True).option('sep',',').option('inferSchema', True).csv('owid-covid-data.csv'))

In [123]:
#Задание 1

df.select(df.iso_code, 'location', f.format_number(df.total_cases_per_million/10000.0, 2)
          .alias('percent')).where(df.date == '2021-03-31').sort(df.total_cases_per_million.desc()).show(15)

+--------+-------------+-------+
|iso_code|     location|percent|
+--------+-------------+-------+
|     AND|      Andorra|  15.54|
|     MNE|   Montenegro|  14.52|
|     CZE|      Czechia|  14.31|
|     SMR|   San Marino|  13.94|
|     SVN|     Slovenia|  10.37|
|     LUX|   Luxembourg|   9.85|
|     ISR|       Israel|   9.63|
|     USA|United States|   9.20|
|     SRB|       Serbia|   8.83|
|     BHR|      Bahrain|   8.49|
|     PAN|       Panama|   8.23|
|     PRT|     Portugal|   8.06|
|     EST|      Estonia|   8.02|
|     SWE|       Sweden|   7.97|
|     LTU|    Lithuania|   7.94|
+--------+-------------+-------+
only showing top 15 rows



In [124]:
#Задание 2

df_new_cases = df.where((df.date <= '2021-03-31') & (df.date >= '2021-03-29') & (f.length(df.iso_code)==3)).groupBy('location','iso_code').sum('new_cases')
df_new_cases = df_new_cases.withColumnRenamed('sum(new_cases)', 'sum_of_new_cases')
df_new_cases.sort(df_new_cases.sum_of_new_cases.desc()).show(15)

+-------------+--------+----------------+
|     location|iso_code|sum_of_new_cases|
+-------------+--------+----------------+
|       Brazil|     BRA|        214059.0|
|United States|     USA|        197717.0|
|        India|     IND|        182021.0|
|       Turkey|     TUR|        109009.0|
|       France|     FRA|         98883.0|
|       Poland|     POL|         70726.0|
|      Germany|     DEU|         58992.0|
|        Italy|     ITA|         52842.0|
|    Argentina|     ARG|         40224.0|
|      Ukraine|     UKR|         30461.0|
|         Iran|     IRN|         29890.0|
|        Spain|     ESP|         29029.0|
|  Philippines|     PHL|         25396.0|
|       Russia|     RUS|         24907.0|
|       Sweden|     SWE|         24868.0|
+-------------+--------+----------------+
only showing top 15 rows



In [148]:
#Задание 3
windowSpec = w.partitionBy(df.iso_code).orderBy(df['date'].asc())
df_delta = df.select('date', 'iso_code', 'new_cases', f.lag(df.new_cases).over(windowSpec)
                     .alias('prev_new_cases')).where((df.date <= '2021-03-31') & (df.date >= '2021-03-29') & (df.iso_code == 'RUS'))
df_delta = df_delta.select('date', 'iso_code', (df_delta.new_cases - df_delta.prev_new_cases).alias('delta'), 'new_cases', 'prev_new_cases')

df_delta.show()

+----------+--------+------+---------+--------------+
|      date|iso_code| delta|new_cases|prev_new_cases|
+----------+--------+------+---------+--------------+
|2021-03-29|     RUS|-390.0|   8589.0|        8979.0|
|2021-03-30|     RUS|-427.0|   8162.0|        8589.0|
|2021-03-31|     RUS|  -6.0|   8156.0|        8162.0|
+----------+--------+------+---------+--------------+

